In [ ]:
# import LGBMClassifier as lgb 

# lgb_model = lgb.LGBMClassifier(bagging_fraction=0.7, bagging_freq=2, boosting_type='gbdt', 
# class_weight=None, colsample_bytree=1.0, feature_fraction=0.8, 
# importance_type='split', learning_rate=0.0005, max_depth=-1, 
# min_child_samples=1, min_child_weight=0.001, min_split_gain=0.2, 
# n_estimators=30, n_jobs=-1, num_leaves=256, objective=None, 
# random_state=123, reg_alpha=3, reg_lambda=0.001, silent='warn', 
# subsample=1.0, subsample_for_bin=200000, subsample_freq=0)


In [ ]:
#tag cell with parameters
project_id =  'divg-josh-pr-d1cc3a'
dataset_id = 'churn_12_months'
resource_bucket = 'divg-josh-pr-d1cc3a-default'
file_bucket = 'divg-josh-pr-d1cc3a-default'
model_id = '5220'
model_name = 'churn_12_months'
pipeline_dataset = 'bq_c12m_training_dataset'
service_type = 'churn_12_months' 

In [ ]:
from google.cloud import bigquery
import logging 
from datetime import datetime
import pandas as pd 
import numpy as np 
    
import os 
import gc
import time
import pickle
import joblib
import logging 
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px

from plotly.subplots import make_subplots
from datetime import datetime
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from google.cloud import storage
from google.cloud import bigquery

from sklearn.metrics import accuracy_score, roc_auc_score, precision_recall_curve, mean_squared_error, f1_score, precision_score, recall_score, confusion_matrix, roc_curve, classification_report

df_train = pd.read_csv('gs://{}/{}/{}_train.csv'.format(file_bucket, service_type, service_type), index_col=False, nrows=10000)  

#train test split
df_train, df_test = train_test_split(df_train, shuffle=True, test_size=0.3, random_state=42,
                                    stratify=df_train['target']
                                    )
#train val split
df_train, df_val = train_test_split(df_train, shuffle=True, test_size=0.3, random_state=42,
                                    stratify=df_train['target']
                                    )

#set up features (list)
cols_1 = df_train.columns.values
cols_2 = df_test.columns.values
cols = set(cols_1).intersection(set(cols_2))
features = [f for f in cols if f not in ['ban', 'target', 'Unnamed: 0']]

create_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

ban_train = df_train['ban']
X_train = df_train[features]
y_train = np.squeeze(df_train['target'].values)

ban_val = df_val['ban']
X_val = df_val[features]
y_val = np.squeeze(df_val['target'].values)

ban_test = df_test['ban']
X_test = df_test[features]
y_test = np.squeeze(df_test['target'].values)



In [ ]:
# Step 1: Get a list of all column names
all_columns = X_train.columns.tolist()

# Step 2: Initialize empty lists for numerical and categorical columns
numerical_features = []
categorical_features = []

# Step 3: Loop through each column to determine its data type
for column in all_columns:
    if pd.api.types.is_numeric_dtype(X_train[column]):
        numerical_features.append(column)
    else:
        categorical_features.append(column)


In [ ]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.ensemble import IsolationForest  # You can also use OneClassSVM or other methods for outlier detection
from sklearn.base import BaseEstimator, TransformerMixin

preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_features),  # Standardization for numerical features
        ('cat', OneHotEncoder(), categorical_features)  # One-hot encoding for categorical features
    ],
    remainder='passthrough'  # Include any unprocessed columns
)

pipeline = Pipeline([
    ('preprocessor', preprocessor)
])


In [ ]:
pipeline.fit(X_train)


In [ ]:
transformed_data = pipeline.transform(X_train)


In [ ]:
type(transformed_data)

In [ ]:
X_train_pp = pd.DataFrame(transformed_data)
X_train_pp.head()

In [ ]:
X_train.head()

In [ ]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.ensemble import IsolationForest  # You can also use OneClassSVM or other methods for outlier detection
from sklearn.base import BaseEstimator, TransformerMixin

def preprocess_ml(df): 
    
    # Step 1: Get a list of all column names
    all_columns = df.columns.tolist()

    # Step 2: Initialize empty lists for numerical and categorical columns
    numerical_features = []
    categorical_features = []

    # Step 3: Loop through each column to determine its data type
    for column in all_columns:
        if pd.api.types.is_numeric_dtype(df[column]):
            numerical_features.append(column)
        else:
            categorical_features.append(column)

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', MinMaxScaler(), numerical_features),  # Standardization for numerical features
            ('cat', OneHotEncoder(), categorical_features)  # One-hot encoding for categorical features
        ],
        remainder='passthrough'  # Include any unprocessed columns
    )

    pipeline = Pipeline([
        ('preprocessor', preprocessor)
    ])

    pipeline.fit(df)

    transformed_df = pipeline.transform(df)

    df_result = pd.DataFrame(transformed_df, columns=df.columns)

    return df_result

In [ ]:
X_train_pp = preprocess_ml(X_train)
print(X_train.shape)
print(X_train_pp.shape)